# Building an Image Classifier in Five Minutes

A common machine learning task is called **image classification**, where the system tries to recognize and label what it sees.   For example, a self-driving car needs to identify things that it sees in its environment, such as other cars, pedestrians and road signs.

Researchers have created and shared machine learning models that are good at recognizing many different categories of objects in images.  These models can be easily re-purposed to recognize a new set of categories through a process called **transfer learning.**  

In this notebook, we will apply transfer learning to re-purpose a machine learning model called a **neural network** to recognize cats and dogs in images.  To teach the model how to categorize images, we need to provide the model with a collection of labeled example images.  Once the model is "trained" to label these example images correctly, it will be able to accurately label any new, unseen images that you give it.

## Downloading the dataset

Here we download and extract a collection of several hundred images of cats and dogs.  The images are organized into a specific directory structure.  The cat images are under `oxford_pets_cat_dog/train/cat` and the dog images are under `oxford_pets_cat_dog/train/dog`.  


In [ ]:
import os
if not os.path.exists('oxford_pets_cat_dog_small.zip'):
  !wget "https://www.dropbox.com/s/pps8tui39ctkha5/oxford_pets_cat_dog_small.zip?dl=1" -O oxford_pets_cat_dog_small.zip
  !unzip -q oxford_pets_cat_dog_small.zip

We specify the path to the dataset in the variable `DATASET_PATH` here.

In [ ]:
#@title Configuration

DATASET_PATH = 'oxford_pets_cat_dog_small' #@param {type: "string"}

This code grabs the names of the class labels from the directory structure.  We will use these class names later when we ask the model to classify images.

In [ ]:
import glob
CLASS_LABELS = ','.join(os.path.basename(dir) for dir in sorted(glob.glob(os.path.join(DATASET_PATH,'train','*'))))
print("Class labels:",CLASS_LABELS)

## Importing the necessary modules

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
!pip install wget
import wget
import imageio
import glob

### Loading the pre-trained neural network

Keras provides many different pre-trained networks.  Here we load a VGG16 network pre-trained on ImageNet.

In [ ]:
vgg = tf.keras.applications.VGG16(include_top=True, weights='imagenet',input_shape=(224,224,3))

The model summary displays the sequence of layers in the model.

In [ ]:
vgg.summary()

This model has been trained to predict 1,000 different object categories.  This is why the last layer has 1000 outputs -- one number for each class.  The class with the highest number is the class chosen by the network for the input image.

We want to re-purpose this model to output two classes.  So, we will remove the last layer and replace it with something else.

In [ ]:
vgg_features = tf.keras.Model(inputs=vgg.input,outputs=vgg.layers[-2].output)

In [ ]:
vgg_features.summary()

### Dataset preparation

Now we will load the dataset and set it up as a Tensorflow dataset.

In [ ]:
builder = tfds.folder_dataset.ImageFolder(DATASET_PATH)
train_ds = builder.as_dataset(split='train', shuffle_files=False)

Here you can see some examples from the dataset with their labels.

In [ ]:
fig = tfds.show_examples(train_ds, builder.info)
fig.show()

We add some processing steps to the dataset:
* Resize images to 224x224
* Preprocess images using the VGG16 preprocess_input() function provided by Keras

In [ ]:
train_ds = train_ds.map(lambda x: {'image':tf.image.resize(x['image'],[224,224]),'label':x['label']})
train_ds = train_ds.map(lambda x: {'image':tf.keras.applications.vgg16.preprocess_input(x['image']),'label':x['label']})
train_ds = train_ds.batch(32)
train_ds = train_ds.prefetch(10)

### Extracting deep features

Now we run the images through the VGG network to extract a 4096-dim vector for each image.  

In [ ]:
train_features = vgg_features.predict(train_ds.map(lambda x: x['image']),verbose=True)

We also put the labels into a Numpy array.

In [ ]:
train_labels = np.concatenate([y for y in train_ds.map(lambda x: x['label'])],axis=0)

Now we have a 4096-dimensional feature vector an integer label for each image.

In [ ]:
train_features.shape, train_labels.shape

### Transfer learning

Now we can train a classifier called a **support vector machine** on our feature vectors and labels.

In [ ]:
import sklearn.svm
classifier = sklearn.svm.SVC().fit(train_features,train_labels)

Let's evaluate on the training set to see how we did.

**Accuracy** means the percentage of images that the classifier correctly labeled.

In [ ]:
from sklearn.metrics import accuracy_score
train_preds = classifier.predict(train_features)
acc = accuracy_score(train_labels,train_preds)
print(f'Accuracy: {acc*100}\%')

Our model classifies over 99% of the dataset correctly!

# Testing on new data

Now you can provide a new image to the classifier, and it will say what it thinks is in the image.

Find the URL for an image of a dog and paste it below.

In [ ]:
TEST_IMAGE_URL = 'https://i.natgeofe.com/n/46b07b5e-1264-42e1-ae4b-8a021226e2d0/domestic-cat_thumb_square.jpg' #@param {type: 'string'}

In [ ]:
filename = wget.download(TEST_IMAGE_URL)

import imageio
image = imageio.imread(filename)
images = image[None,...]

images_resized = tf.image.resize(images,[224,224])
images_pre = tf.keras.applications.vgg16.preprocess_input(images_resized)

features = vgg_features.predict(images_pre)
preds = classifier.predict(features)

class_labels = CLASS_LABELS.split(',')
pred_label = class_labels[preds[0]]

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(image)
plt.title(f'classification: {pred_label}')
plt.show()

# Experiment

Now it's your turn!  Make your own dataset and train the image classifier to recognize objects in it.

You should first make a copy of this notebook so that you can save your changes.  Go to File > Save a Copy in Drive.

Then, connect to Google Drive using the button in the "Files" panel on the left.  Pick two categories and assemble a small dataset of images in the proper directory structure in your Google Drive.  For example, you could find ten pictures of apples and ten pictures of oranges using [Google Images](https://images.google.com/), and put them into directories called "fruits/train/apple" and "fruits/train/orange".

Set the `DATASET_PATH` variable to the location of your dataset on Google Drive (e.g. `drive/MyDrive/fruits`) and run through the notebook.  In the last step, provide the URL of a new image that wasn't included in your training set.  Does the model properly recognize the image?

